Coursera assignment: Toronto Neighborhoods - FULL (Parts 1 + 2 +3)

Part 1: I use pandas to extract a table from a wikipedia page into a data frame, which I then clean

Note: in the first few code blocks, I import all the libraries I need for all three parts, based on the New York assignment we studied in the course

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json5 # library to handle JSON files

In [2]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [3]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [5]:
import folium # map rendering library

In [6]:
print('Libraries imported.')

Libraries imported.


In [7]:
# display options so as to view data frames in full
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [9]:
dfs=pd.read_html(url, header=0) # read data frames from the designated url (there might be more than one)

There's more than one data frame on the page, but I only want the first one, so I loop through them and store the first

In [10]:
x=0
for df in dfs:
    if x==0:
        nbdf=df
    x=x+1

In [11]:
print(nbdf)

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25          M8C      Not assigned   
2

I will now create a data frame that excludes the rows where the Borough is Not assigned

In [12]:
nbdf2=nbdf[nbdf['Borough']!='Not assigned']
print(nbdf2)

    Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
8           M9A         Etobicoke   
9           M1B       Scarborough   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
17          M9B         Etobicoke   
18          M1C       Scarborough   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
26          M9C         Etobicoke   
27          M1E       Scarborough   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
36          M1G       Scarborough   
39          M4G         East York   
40          M5G  Downtown Toronto   
41          M6G  Downtown Toronto   
4

178  Mimico NW, The Queensway West, South of Bloor,...  


I will now check whether 'Not assigned' appears under the Neighborhood column - if not I'll get an empty data frame 

In [13]:
print(nbdf2[nbdf2['Neighborhood']=='Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


In the assignment instructions, it is stated that there are duplicate postal codes, however I show this is not the case

In [14]:
nbdf2.duplicated(['Postal Code']).any()

False

There are no remaining cases of either the Borough or the Neighboorhood being 'Not assigned', and there are no
duplicate postal codes. I will now display the number of remaining rows as per the instructions

In [15]:
nbdf2.shape

(103, 3)

The first part of the assignment has been completed

Part 2: get the GPS coordinates and insert them correctly into the neighborhoods data frame

In [16]:
url_gps='http://cocl.us/Geospatial_data'

In [17]:
df_gps=pd.read_csv(url_gps)
df_gps.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now I need to merge the neighborhoods data frame with the gps data frame, matching on the Postal Code column

In [18]:
merged_df = nbdf2.merge(df_gps, how = 'inner', on = ['Postal Code'])
merged_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


This concludes Part 2 of the assignment

Part 3: carry out a clustering analysis, using data about venues in each neighbourhood from Foursquare

I start by creating a general map of the Toronto area, with blue points indicating each neighborhood

To ensure the map is well centered, I compute and use the mean latitude and mean longitude from the data frame

In [19]:
# compute central coordinates for the map based on mean coordinates from the data frame
tor_lat=merged_df['Latitude'].mean()
tor_long=merged_df['Longitude'].mean()
print(tor_lat,tor_long)

43.704607733980595 -79.3971529116505


In [20]:
# create map of Toronto
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto.save("map1.html") # There are difficulties with map display so I save to html instead 
print("Map1 saved")
map_toronto

Map1 saved


I choose to focus on quite central neighborhoods. I pick the neighborhood that includes Union Station as a reference.

In [21]:
centre=merged_df[merged_df['Neighborhood'].str.contains('Union Station')]
centre

,Postal Code,Borough,Neighborhood,Latitude,Longitude
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


I will use the Haversine distance function to compute distances from each neighborhood to the central neighborhood, in km

In [22]:
# source of code
# https://towardsdatascience.com/heres-how-to-calculate-distance-between-2-geolocations-in-python-93ecab5bbba4

def haverdist(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

p1lat=float(centre['Latitude'])
p1long=float(centre['Longitude'])

def haver_df(row):
    p2lat=float(row['Latitude'])
    p2long=float(row['Longitude'])
    dist=haverdist(p1lat,p1long,p2lat,p2long)
    return dist

merged_df['Distance_Center']=merged_df.apply (lambda row: haver_df(row), axis=1)

merged_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Distance_Center
0,M3A,North York,Parkwoods,43.753259,-79.329656,13.19
1,M4A,North York,Victoria Village,43.725882,-79.315572,10.85
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.26
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,10.92
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.47
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,12.47
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,23.81
7,M3B,North York,Don Mills,43.745906,-79.352188,11.92
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,9.30
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.83


Now I define a new, smaller data frame, with only those neighborhoods that are less than 4 miles from the center

Of course one could change the radius and capture more, or fewer, neighborhoods

In [23]:
radius_miles=4
radius=1.609*radius_miles # convert to km
central_toronto=merged_df[merged_df['Distance_Center']<radius].reset_index()
central_toronto

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Distance_Center
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.26
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.47
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.83
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1.29
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.81
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.96
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.58
7,30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1.11
8,31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,5.79
9,35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,6.07


In [24]:
# compute new central coordinates for a new map for the reduced data frame
tor_lat2=central_toronto['Latitude'].mean()
tor_long2=central_toronto['Longitude'].mean()
print(tor_lat2,tor_long2)

43.659514625806445 -79.38384580322582


In [25]:
# create new map - central Toronto
map_toronto2 = folium.Map(location=[tor_lat2, tor_long2], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Borough'], central_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2.save("map2.html") # There are difficulties with map display so I save to html too 
print("Map2 saved")
map_toronto2

Map2 saved


Now I will use the Foursquare API to get venue information about various neighborhoods

First step: define credentials

In [26]:
CLIENT_ID = 'ZO3BBSIQNBQP3WG1YW0X0JUB0GCG0BTCRRBIZLALIWTTQK1X'
CLIENT_SECRET = 'S0GMZOANBJLCXIJMUCK0MPRMYLNNBBEHDVJWS5QLJHBHIL50'
VERSION = '20180605' # Foursquare API version

In [27]:
# as in the course example, define max no of venues to be extracted per request, and define geog. radius of requests
LIMIT=100
radius_4sq=400

In [28]:
def getNearbyVenues(names, latitudes, longitudes, rad=radius_4sq):
    
    venues_list=[]
    print("Processing each neighborhood...")
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
print("Getting venue information by neighbourhood")
torvenues = getNearbyVenues(names=central_toronto['Neighborhood'],
                                   latitudes=central_toronto['Latitude'],
                                   longitudes=central_toronto['Longitude']
                                  )
print("Venue information has been obtained!")

Getting venue information by neighbourhood
Processing each neighborhood...
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
University of Toronto, Harbord
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, 

In [30]:
print(torvenues.shape)
torvenues.head(15)

(1184, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
7,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
8,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
9,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


In [31]:
#torvenues.groupby('Neighborhood').count()

In [32]:
print('There are {} uniques categories.'.format(len(torvenues['Venue Category'].unique())))

There are 203 uniques categories.


In [33]:
# one hot encoding
toronehot = pd.get_dummies(torvenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronehot['Neighborhood'] = torvenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronehot.columns[-1]] + list(toronehot.columns[:-1])
toronehot = toronehot[fixed_columns]

toronehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [34]:
torgrouped = toronehot.groupby('Neighborhood').mean().reset_index()
torgrouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.055556,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.055556,0.111111,0.055556,0.00,0.00,0.00,0.000000,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Define function to sort venues in descending order

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue Type'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torgrouped['Neighborhood']

for ind in np.arange(torgrouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torgrouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(12)

,Neighborhood,1st Most Common Venue Type,2nd Most Common Venue Type,3rd Most Common Venue Type,4th Most Common Venue Type,5th Most Common Venue Type,6th Most Common Venue Type,7th Most Common Venue Type
0,Berczy Park,Cocktail Bar,Greek Restaurant,Sushi Restaurant,Concert Hall,Comfort Food Restaurant,Grocery Store,Coffee Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Italian Restaurant,Grocery Store,Office
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Park,Restaurant,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport Lounge,Plane,Coffee Shop,Rental Car Location,Bar,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Chinese Restaurant,Japanese Restaurant,Bank,Sushi Restaurant
5,Christie,Café,Grocery Store,Baby Store,Coffee Shop,Italian Restaurant,Nightclub,Candy Store
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Smoke Shop,Men's Store
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Gym
8,"Dufferin, Dovercourt Village",Bakery,Portuguese Restaurant,Bar,Music Venue,Bank,Middle Eastern Restaurant,Park
9,"East Toronto, Broadview North (Old East York)",Convenience Store,Park,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space


And now the cluster analysis using k-means

In [37]:
# set number of clusters
kclusters = 5

torgrouped_clustering = torgrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torgrouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 0, 2, 2, 2, 2, 3])

Create a new dataframe that includes the cluster as well as the top 10 venue types for each neighborhood

Note: contrary to the New York lab we did in the course, I had a problem with missing data for one of the neighborhoods

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

final_df = central_toronto

# merge toronto_grouped with central_toronto to add latitude/longitude for each neighborhood
final_df = final_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# get rid of rows (neighborhoods) for which data was not available
final_df.dropna(axis=0,inplace=True)

final_df

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Distance_Center,Cluster Labels,1st Most Common Venue Type,2nd Most Common Venue Type,3rd Most Common Venue Type,4th Most Common Venue Type,5th Most Common Venue Type,6th Most Common Venue Type,7th Most Common Venue Type
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.26,0.0,Coffee Shop,Breakfast Spot,Health Food Store,Spa,Bakery,Bank,Distribution Center
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.47,0.0,Coffee Shop,Sushi Restaurant,Distribution Center,Park,Fried Chicken Joint,Bank,Italian Restaurant
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.83,2.0,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Bar,Hotel,Italian Restaurant
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1.29,2.0,Gastropub,Coffee Shop,Cocktail Bar,Café,Restaurant,Pharmacy,Camera Store
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.81,2.0,Cocktail Bar,Greek Restaurant,Sushi Restaurant,Concert Hall,Comfort Food Restaurant,Grocery Store,Coffee Shop
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.96,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Chinese Restaurant,Japanese Restaurant,Bank,Sushi Restaurant
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.58,2.0,Café,Grocery Store,Baby Store,Coffee Shop,Italian Restaurant,Nightclub,Candy Store
7,30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1.11,2.0,Coffee Shop,Café,Sushi Restaurant,Salad Place,Japanese Restaurant,Steakhouse,Thai Restaurant
8,31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,5.79,2.0,Bakery,Portuguese Restaurant,Bar,Music Venue,Bank,Middle Eastern Restaurant,Park
9,35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,6.07,3.0,Convenience Store,Park,Women's Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space


Create final map with clusters

In [39]:
# create map - same as the map of central Toronto I made before
map_clusters = folium.Map(location=[tor_lat2, tor_long2], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_df['Latitude'], final_df['Longitude'], final_df['Neighborhood'], final_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save("map3.html")
print("map3 saved")
map_clusters

map3 saved
